<a href="https://colab.research.google.com/github/BiniCodes/caffeine_coders_emotion_detection/blob/Datasets/Combine_training_and_testing_dataset_for_bigger_accuracy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip3 install http://download.pytorch.org/whl/cu80/torch-0.4.1-cp36-cp36m-linux_x86_64.whl 

     |████████████████████████████████| 483.0MB 1.1MB/s 
ERROR: torchvision 0.3.0 has requirement torch>=1.1.0, but you'll have torch 0.4.1 which is incompatible.
ERROR: fastai 1.0.57 has requirement torch>=1.0.0, but you'll have torch 0.4.1 which is incompatible.


In [0]:
import torch
import torch.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence


In [0]:
import re
import numpy as np
import time
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

import itertools
import pandas as pd
from scipy import stats
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
#!cp '/content/gdrive/My Drive/Colab Notebooks/Main Project Emotion Recognition/TrainingData/fear_TrainingData.csv' 'fearTra.csv'

In [0]:
### Helper functions
import pickle

def convert_to_pickle(item, directory):
    pickle.dump(item, open(directory,"wb"))


def load_from_pickle(directory):
    return pickle.load(open(directory,"rb"))

In [0]:
trainingData = load_from_pickle("/content/gdrive/My Drive/Colab Notebooks/Main Project Emotion Recognition/TrainingData/cleanTrainingData1.pkl")

In [0]:
testingData = load_from_pickle("/content/gdrive/My Drive/Colab Notebooks/Main Project Emotion Recognition/TestingData/cleanTestingData1.pkl")

In [0]:
trainingData.shape


(3613, 4)

In [0]:
testingData.shape

(9426, 4)

In [0]:
data = pd.concat([testingData, trainingData])

In [0]:
data.shape



(13039, 4)

In [0]:
import os
import glob

os.chdir("/content/gdrive/My Drive/Colab Notebooks/Main Project Emotion Recognition/")

In [0]:
data.to_pickle("./cleanedDATA.pkl")

In [0]:
data = pd.read_pickle("./cleanedDATA.pkl")

In [0]:
data

,id,tweets,emotion,intensity
0,10941,at the point today where if someone says somet...,anger,0.319
1,10942,it s game day t minus relentless,anger,0.144
2,10943,this game has pissed me off more than any othe...,anger,0.898
3,10944,i ve just found out it s candice and not canda...,anger,0.271
4,10945,if he can t come to my mum a th after k tweets...,anger,0.646
5,10946,if he can t come to my mum a th after k tweets...,anger,0.583
6,10947,wanna go home and focus up on this game don t ...,anger,0.375
7,10948,i ve been disconnected whilst on holiday but i...,anger,0.625
8,10949,i ve been disconnected whilst on holiday but i...,anger,0.396
9,10950,i wanna see you smile i don t wanna see you ma...,anger,0.250


In [0]:
load_from_pickle("/content/gdrive/My Drive/Colab Notebooks/Main Project Emotion Recognition/cleanedDATA.pkl")

,id,tweets,emotion,intensity
0,10941,at the point today where if someone says somet...,anger,0.319
1,10942,it s game day t minus relentless,anger,0.144
2,10943,this game has pissed me off more than any othe...,anger,0.898
3,10944,i ve just found out it s candice and not canda...,anger,0.271
4,10945,if he can t come to my mum a th after k tweets...,anger,0.646
5,10946,if he can t come to my mum a th after k tweets...,anger,0.583
6,10947,wanna go home and focus up on this game don t ...,anger,0.375
7,10948,i ve been disconnected whilst on holiday but i...,anger,0.625
8,10949,i ve been disconnected whilst on holiday but i...,anger,0.396
9,10950,i wanna see you smile i don t wanna see you ma...,anger,0.250
